#### 导入第三方库

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import itertools

#### 实例化模块

In [2]:
# Processing_time：工件各工序对应各机器加工时间矩阵
# J：各工件对应的工序数字典
# M_num：加工机器数
# O_num：加工工序数
# J_num：工件个数

CKS201 = [[10, 9, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 14, 16, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 15, 25, 21, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 9, 13, 15, 24, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 10]]
CKS301 = [[12, 9, 10, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 16, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 15, 9999, 9999, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 9999, 27, 22, 9999, 9999, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 21, 17, 9999, 9999, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 19, 9999, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 17, 9999],
          [9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 18]]
Processing_time = [CKS201]*3
Processing_time.extend([CKS301]*4)
J = {i:5 for i in range(1,4)}
J.update({j:8 for j in range(4,8)})
M_num = 12
O_num = 3 * 5 + 4 * 8
J_num = 7

#### 工件处理模块

In [3]:
class Job:
    def __init__(self, Job_index, Operation_num):
        self.Job_index = Job_index # 工件序号
        self.Operation_num = Operation_num  # 工序数
        self.Processed = [] # 记录工件工序的加工进度
        self.J_start = [] # 记录工件工序的开始时间
        self.J_end = [] # 记录工件工序的结束时间
        self.J_machine = [] # 记录工件工序选择的机器
        self.Last_Processing_Machine = None # 工件当前工序的加工机器
        self.Last_Processing_end_time = 0 # 工件当前工序的结束时间

    def Current_Processed(self):
        return len(self.Processed)

    def _Input(self, W_Eailiest, End_time, Machine): # 工件当前工序的开始时间、结束时间、选择加工机器
        self.Last_Processing_Machine = Machine
        self.Last_Processing_end_time = End_time
        self.Processed.append(1)
        self.J_start.append(W_Eailiest)
        self.J_end.append(End_time)
        self.J_machine.append(Machine)

#### 机器处理模块

In [4]:
class Machine_Time_window:
    def __init__(self, Machine_index):
        self.Machine_index = Machine_index # 加工机器序号
        self.assigned_task = [] # 机器分配的任务记录，包括工件序号以及工序序号
        self.O_start = [] # 各任务工序的开始时间记录
        self.O_end = [] # 各任务工序的结束时间记录
        self.End_time = 0

    # 机器的哪些时间窗是空的,此处只考虑内部封闭的时间窗,类似甘特图每一行往后叠加
    def Empty_time_window(self):
        time_window_start = []
        time_window_end = []
        len_time_window = []
        if self.O_end is None:
            pass
        elif len(self.O_end) == 1:
            if self.O_start[0] != 0:
                time_window_start = [0]
                time_window_end = [self.O_start[0]]
        elif len(self.O_end) > 1:
            if self.O_start[0] != 0:
                time_window_start.append(0)
                time_window_end.append(self.O_start[0])
            time_window_start.extend(self.O_end[:-1])  # 因为使用时间窗的结束点就是空时间窗的开始点
            time_window_end.extend(self.O_start[1:])
        if time_window_end is not None:
            len_time_window = [time_window_end[i] - time_window_start[i] for i in range(len(time_window_end))]
        return time_window_start, time_window_end, len_time_window  #空格时间的开始，结束，时长

    # 机器投入新一轮加工
    def _Input(self, Job, M_Ealiest, P_t, O_num):
        if self.O_end != []:
            # 如果当前机器加工的最早开始时间比记录的大，则依次往后排任务，否则将任务插入中间的分配任务记录
            if self.O_start[-1] > M_Ealiest:
                for i in range(len(self.O_end)):
                    if self.O_start[i] >= M_Ealiest:
                        self.assigned_task.insert(i, [Job + 1, O_num + 1])
                        break
            else:
                self.assigned_task.append([Job + 1, O_num + 1])
        else:
            self.assigned_task.append([Job + 1, O_num + 1])
        self.O_start.append(M_Ealiest)
        self.O_start.sort()
        self.O_end.append(M_Ealiest + P_t)
        self.O_end.sort()
        self.End_time = self.O_end[-1]

#### 基因编码模块

In [5]:
class Encode:
    def __init__(self, Matrix, Pop_size, J, J_num, M_num, O_num): # Pop_size 种群数量
        self.Matrix = Matrix  # 工件各工序对应各机器加工时间矩阵
        self.Pop_size = Pop_size # 种群数量
        self.J = J  # 各工件对应的工序数
        self.J_num = J_num  # 工件数
        self.M_num = M_num  # 机器数
        self.O_num = O_num  # 工序数

    # 生成工序准备的部分，便于后续基因的工序排序部分编码
    def OS_List(self):
        OS_list = []
        for k, v in self.J.items():
            OS_add = [k - 1 for j in range(v)]
            OS_list.extend(OS_add)
        return OS_list

    # 种群初始化
    def Initial(self):
        MS = np.zeros([self.Pop_size, self.O_num], dtype=int)
        OS_list = self.OS_List()
        OS = np.zeros([self.Pop_size, self.O_num], dtype=int)
        for i in range(self.Pop_size):
            # 生成基因的工序排序部分
            random.shuffle(OS_list)
            OS_gongxu = OS_list
            OS[i] = np.array(OS_gongxu)
            # 生成基因的机器选择部分
            GJ_list = [j for j in range(self.J_num)]
            A = 0
            for gon in GJ_list:
                g = gon  # 随机选择工件集的第一个工件并从工件集中剔除这个工件
                h = np.array(self.Matrix[g])  # 第一个工件及其对应工序的加工时间
                for j in range(len(h)):  # 从工件的第一个工序开始选择机器
                    D = np.array(h[j])
                    List_Machine_weizhi = []
                    Site = 0
                    for k in range(len(D)):  # 每道工序可使用的机器以及机器的加工时间
                        if D[k] == 9999:  # 确定可加工该工序的机器
                            continue
                        else:
                            List_Machine_weizhi.append(Site)
                            Site += 1
                    Machine_Index_add = random.choice(List_Machine_weizhi)
                    MS[i][A] = MS[i][A] + Machine_Index_add
                    A += 1
        CHS = np.hstack((MS, OS))
        return CHS  # 种群基因

#### 基因解码模块

In [6]:
class Decode:
    def __init__(self, J, Processing_time, M_num):
        self.Processing_time = Processing_time # 工件各工序对应各机器加工时间矩阵
        self.Scheduled = []  # 已经排产过的工序
        self.M_num = M_num  # 加工机器数
        self.Machines = []  # 存储机器类
        self.fitness = 0
        self.J = J   # 各工件对应的工序数字典
        for j in range(M_num):
            self.Machines.append(Machine_Time_window(j))
        self.Machine_State = np.zeros(M_num, dtype=int)  # 在机器上加工的工件是哪个
        self.Jobs = []  # 存储工件类
        for k, v in J.items():
            self.Jobs.append(Job(k, v))

    # 时间顺序矩阵和机器顺序矩阵，根据基因的MS部分转换
    def Order_Matrix(self, MS):
        JM = []
        T = []
        Ms_decompose = []
        Site = 0
        # 按照基因的MS部分按工件序号划分
        for S_i in self.J.values():
            Ms_decompose.append(MS[Site:Site + S_i])
            Site += S_i
        for i in range(len(Ms_decompose)):  # len(Ms_decompose)表示工件数
            JM_i = []
            T_i = []
            for j in range(len(Ms_decompose[i])): # len(Ms_decompose[i])表示每一个工件对应的工序数
                O_j = self.Processing_time[i][j] # 工件i的工序j可选择的加工时间列表
                M_ij = []
                T_ij = []
                for Mac_num in range(len(O_j)):  # 寻找MS对应部分的机器时间和机器顺序
                    if O_j[Mac_num] != 9999:
                        M_ij.append(Mac_num)
                        T_ij.append(O_j[Mac_num])
                    else:
                        continue
                JM_i.append(M_ij[Ms_decompose[i][j]])
                T_i.append(T_ij[Ms_decompose[i][j]])
            JM.append(JM_i)
            T.append(T_i)
        return JM, T

    # 确定工序的最早加工时间
    def Earliest_Start(self, Job, O_num, Machine):
        P_t = self.Processing_time[Job][O_num][Machine]
        last_O_end = self.Jobs[Job].Last_Processing_end_time  # 上道工序结束时间
        Selected_Machine = Machine
        M_window = self.Machines[Selected_Machine].Empty_time_window() #当前机器的空格时间
        M_Tstart = M_window[0]
        M_Tend = M_window[1]
        M_Tlen = M_window[2]
        Machine_end_time = self.Machines[Selected_Machine].End_time
        ealiest_start = max(last_O_end, Machine_end_time)
        if M_Tlen is not None:  # 此处为全插入时窗
            for le_i in range(len(M_Tlen)):
                # 当前空格时间比加工时间大可插入
                if M_Tlen[le_i] >= P_t:
                    # 当前空格开始时间比该工件上一工序结束时间大可插入该空格，以空格开始时间为这一工序开始
                    if M_Tstart[le_i] >= last_O_end:
                        ealiest_start = M_Tstart[le_i]
                        break
                    # 当前空格开始时间比该工件上一工序结束时间小但空格可满足插入该工序，以该工序的上一工序的结束为开始
                    if M_Tstart[le_i] < last_O_end and M_Tend[le_i] - last_O_end >= P_t:
                        ealiest_start = last_O_end
                        break
        M_Ealiest = ealiest_start  # 当前工件当前工序的最早开始时间
        End_work_time = M_Ealiest + P_t # 当前工件当前工序的结束时间
        return M_Ealiest, Selected_Machine, P_t, O_num, last_O_end, End_work_time

    # 解码
    def decode(self, CHS, Len_Chromo): # CHS：种群基因  Len_Chromo：MS与OS的分解线
        MS = list(CHS[0:Len_Chromo])
        OS = list(CHS[Len_Chromo:2 * Len_Chromo])
        Needed_Matrix = self.Order_Matrix(MS)
        JM = Needed_Matrix[0]
        for i in OS:
            Job = i
            O_num = self.Jobs[Job].Current_Processed() # 现在加工的工序
            Machine = JM[Job][O_num]  # 用基因的OS部分的工件序号以及工序序号索引机器顺序矩阵的机器序号
            Para = self.Earliest_Start(Job, O_num, Machine)
            self.Jobs[Job]._Input(Para[0], Para[5], Para[1]) # 工件完成该工序
            if Para[5] > self.fitness:
                self.fitness = Para[5]
            self.Machines[Machine]._Input(Job, Para[0], Para[2], Para[3]) # 机器完成该工件该工序
        return self.fitness

#### 遗传算法流程模块

In [7]:
class GA():
    def __init__(self):
        self.Pop_size = 400  # 种群数量
        self.Pc = 0.8   # 交叉概率
        self.Pm = 0.3   # 变异概率
        self.Pv = 0.5   # 选择何种方式进行交叉的概率阈值
        self.Pw = 0.95  # 选择何种方式进行变异的概率阈值
        self.Max_Itertions = 100  # 最大迭代次数

    # 适应度
    def fitness(self, CHS, J, Processing_time, M_num, Len):
        Fit = []
        for i in range(len(CHS)):
            d = Decode(J, Processing_time, M_num)
            Fit.append(d.decode(CHS[i], Len))
        return Fit

    # 机器部分交叉
    # CHS：基因
    # T0：工序总数
    def machine_cross(self, CHS1, CHS2, T0):
        T_r = [j for j in range(T0)]
        r = random.randint(1, 10)  # 在区间[1,T0]内产生一个整数r
        random.shuffle(T_r)
        R = T_r[0:r]  # 按照随机数r产生r个互不相等的整数
        OS_1 = CHS1[O_num:2 * T0]
        OS_2 = CHS2[O_num:2 * T0]
        MS_1 = CHS2[0:T0]
        MS_2 = CHS1[0:T0]
        for i in R:
            K, K_2 = MS_1[i], MS_2[i]
            MS_1[i], MS_2[i] = K_2, K
        CHS1 = np.hstack((MS_1, OS_1))
        CHS2 = np.hstack((MS_2, OS_2))
        return CHS1, CHS2

    # 工序部分交叉
    # CHS：基因
    # T0：工序总数
    # J_num：工件总数
    def operation_cross(self, CHS1, CHS2, T0, J_num):
        OS_1 = CHS1[T0:2 * T0]
        OS_2 = CHS2[T0:2 * T0]
        MS_1 = CHS1[0:T0]
        MS_2 = CHS2[0:T0]
        Job_list = [i for i in range(J_num)]
        random.shuffle(Job_list)
        r = random.randint(1, J_num - 1)
        Set1 = Job_list[0:r]
        Set2 = Job_list[r:J_num]
        new_os = list(np.zeros(T0, dtype=int))
        for k, v in enumerate(OS_1):
            if v in Set1:
                new_os[k] = v + 1
        for i in OS_2:
            if i not in Set1:
                Site = new_os.index(0)
                new_os[Site] = i + 1
        new_os = np.array([j - 1 for j in new_os])
        CHS1 = np.hstack((MS_1, new_os))
        CHS2 = np.hstack((MS_2, new_os))
        return CHS1, CHS2

    # 机器部分变异
    # CHS：基因
    # O：加工时间矩阵
    # T0：工序总数
    # J：各工件加工信息
    def machine_variation(self, CHS, O, T0, J):
        Tr = [i_num for i_num in range(T0)]
        MS = CHS[0:T0]
        OS = CHS[T0:2 * T0]
        # 机器选择部分
        r = random.randint(1, T0 - 1)  # 在变异染色体中选择r个位置
        random.shuffle(Tr)
        T_r = Tr[0:r]
        for num in T_r:
            T_0 = [j for j in range(T0)]
            K = []
            Site = 0
            for k, v in J.items():
                K.append(T_0[Site:Site + v])
                Site += v
            for i in range(len(K)):
                if num in K[i]:
                    O_i = i
                    O_j = K[i].index(num)
                    break
            Machine_using = O[O_i][O_j]
            Machine_time = []
            for j in Machine_using:
                if j != 9999:
                    Machine_time.append(j)
            Min_index = Machine_time.index(min(Machine_time))
            MS[num] = Min_index
        CHS = np.hstack((MS, OS))
        return CHS

    # 工序部分变异
    # CHS：基因
    # T0：工序总数
    # J_num：工件总数
    # J_：各工件加工信息
    # O：加工时间矩阵
    # M_num：机器总数
    def operation_variation(self, CHS, T0, J_num, J, O, M_num):
        MS = CHS[0:T0]
        OS = list(CHS[T0:2 * T0])
        r = random.randint(1, J_num - 1)
        Tr = [i for i in range(J_num)]
        random.shuffle(Tr)
        Tr = Tr[0:r]
        J_os = dict(enumerate(OS))  # 随机选择r个不同的基因
        J_os = sorted(J_os.items(), key=lambda d: d[1])
        Site = []
        for i in range(r):
            Site.append(OS.index(Tr[i]))
        A = list(itertools.permutations(Tr, r))
        A_CHS = []
        for i in range(len(A)):
            for j in range(len(A[i])):
                OS[Site[j]] = A[i][j]
            C_I = np.hstack((MS, OS))
            A_CHS.append(C_I)
        Fit = []
        for i in range(len(A_CHS)):
            d = Decode(J, O, M_num)
            Fit.append(d.decode(CHS, T0))
        return A_CHS[Fit.index(min(Fit))]

#### 甘特图绘制模块

In [8]:
def Gantt(Machines):
    M = ['red', 'blue', 'yellow', 'orange', 'green', 'palegoldenrod', 'purple', 'pink', 'Thistle', 'Magenta',
         'SlateBlue', 'RoyalBlue', 'Cyan', 'Aqua', 'floralwhite', 'ghostwhite', 'goldenrod', 'mediumslateblue',
         'navajowhite', 'navy', 'sandybrown', 'moccasin']
    for i in range(len(Machines)):
        Machine = Machines[i]
        Start_time = Machine.O_start
        End_time = Machine.O_end
        for i_1 in range(len(End_time)):
            plt.barh(i, width=End_time[i_1] - Start_time[i_1], height=0.8, left=Start_time[i_1],
                     color=M[Machine.assigned_task[i_1][0] - 1], edgecolor='black')
            plt.text(x=Start_time[i_1] + (End_time[i_1] - Start_time[i_1]) / 2 - 0.5, y=i,
                     s=Machine.assigned_task[i_1][0])
    plt.yticks(np.arange(len(Machines) + 1), np.arange(1, len(Machines) + 2))
    plt.title('Scheduling Gantt chart')
    plt.ylabel('Machines')
    plt.xlabel('Time(min)')
    plt.savefig('优化后排程方案的甘特图.png')
    plt.show()

#### 主文件

In [ ]:
if __name__ == '__main__':
    Optimal_fit = 9999      # 最佳适应度（初始化）
    Optimal_CHS = 0         # 最佳适应度对应的基因个体（初始化）
    g = GA()
    e = Encode(Processing_time, g.Pop_size, J, J_num, M_num, O_num)
    C = e.Initial()
    Best_fit = []  # 记录适应度在迭代过程中的变化，便于绘图
    for i in range(g.Max_Itertions):
        print("iter_{} start!".format(i))
        Fit = g.fitness(C, J, Processing_time, M_num, O_num)
        Best = C[Fit.index(min(Fit))]
        best_fitness = min(Fit)
        if best_fitness < Optimal_fit:
            Optimal_fit = best_fitness
            Optimal_CHS = Best
            Best_fit.append(Optimal_fit)
            print('best_fitness', best_fitness)
            d = Decode(J, Processing_time, M_num)
            Fit.append(d.decode(Optimal_CHS, O_num))
            Gantt(d.Machines)
        else:
            Best_fit.append(Optimal_fit)
        for j in range(len(C)):
            Cafter = []
            if random.random() < g.Pc:
                N_i = random.choice(np.arange(len(C)))
                if random.random() < g.Pv:
                    Cross = g.machine_cross(C[j], C[N_i], O_num)
                else:
                    Cross = g.operation_cross(C[j], C[N_i], O_num, J_num)
                Cafter.append(Cross[0])
                Cafter.append(Cross[1])
                Cafter.append(C[j])
            if random.random() < g.Pm:
                if random.random() < g.Pw:
                    vvv = g.machine_variation(C[j], Processing_time, O_num, J)
                else:
                    vvv = g.operation_variation(C[j], O_num, J_num, J, Processing_time, M_num)
                Cafter.append(vvv)
            if Cafter != []:
                Fit = g.fitness(Cafter, J, Processing_time, M_num, O_num)
                C[j] = Cafter[Fit.index(min(Fit))]
    # 绘制甘特图
    x = np.linspace(0, 50, g.Max_Itertions)
    plt.plot(x, Best_fit, '-k')
    plt.title('the maximum completion time of each iteration')
    plt.ylabel('Cmax')
    plt.xlabel('Test Num')
    plt.savefig('最大完成时间的优化过程.png')
    plt.show()